# Geographic features from Hathi volumes

Create geographical similarity features in a list of volumes identified by HTIDs. Methods of comparison and number of output features customizable.

## Imports and setup

Note that if we're using live data from the Textual Geographies database, we need to first establish a port-forwarding SSH connection:

```
ssh mwilkens@geolit.crc.nd.edu -L 5433:localhost:5433
```

Can skip this if pulling data from local files (set variable `use_db=False` in the first code block below).

In [1]:
# Imports
import csv
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from   sklearn.decomposition import PCA
from   sklearn.feature_extraction.text import TfidfVectorizer
from   sklearn.preprocessing import StandardScaler
from   IPython.display import display
import string 

# Variables and options
input_dir = 'inputs'
results_dir = 'results'
fig_dir = 'figures'
use_db = False # Pull live data from Postgres? (else use local CSV)
metadata_filename = 'rh.csv'
postgres_credentials = '/Users/mwilkens/Google Drive/Private/postgresql-96-credentials.txt'
postgres_server      = 'localhost'
postgres_db_name     = 'research'
postgres_port        = '5433'

# Setup
%matplotlib inline
metadata_file = os.path.join(input_dir, metadata_filename)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(fig_dir, exist_ok=True)
sns.set()
sns.set_context('talk')
plt.rc('figure', figsize=(12, 8))

## Load metadata

In [2]:
# Read in corpus metadata
meta = pd.read_csv(metadata_file, sep=',')

# Basic stats on corpus counts and publication dates
display(meta.head())
display(meta.describe())
display(meta.info())

,TITLE,AUTHOR,HATHI_ID,PUBLISHER,YEAR,GENDER,RACE,NATIONALITY,GENRE,YEAR_CORRECT
0,A Christmas Book,"Anglund, Joan Walsh",NaN,Random House,1950,F,Caucasian,American,LIT,1983
1,"20,000 Baseball Cards Under The Sea","Buller, Jon",NaN,Random House,1950,M,Caucasian?,American,YA,1991
2,A Christmas Memory,"Capote, Truman",NaN,Random House,1950,M,Caucasian,American,LIT,1956
3,7Th Heaven,"Christie, Amanda",NaN,Random House,1950,F,white,American,ROM,2000
4,20Th-Century Dreams,"Cohn, Nik",NaN,Random House,1950,M,Caucasian,British,LIT,1999


,TITLE,AUTHOR,HATHI_ID,PUBLISHER,YEAR,GENDER,RACE,NATIONALITY,GENRE,YEAR_CORRECT
count,3525,3506,1371,3525,3525,3425,3309,3341,3503,3524
unique,3525,2023,1371,348,111,16,27,100,13,142
top,"A man to conjure with, | a novel.","Osborne, Mary Pope",uc1.32106002180732,Random House,2000,M,Caucasian,American,LIT,2000
freq,1,50,1,2154,180,1359,2068,1771,995,228


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3525 entries, 0 to 3524
Data columns (total 10 columns):
TITLE           3525 non-null object
AUTHOR          3506 non-null object
HATHI_ID        1371 non-null object
PUBLISHER       3525 non-null object
YEAR            3525 non-null object
GENDER          3425 non-null object
RACE            3309 non-null object
NATIONALITY     3341 non-null object
GENRE           3503 non-null object
YEAR_CORRECT    3524 non-null object
dtypes: object(10)
memory usage: 275.5+ KB


None

Lots of cleanup on the metadata that could happen here, but we only need the list of HTIDs, so not doing anything else.

Get list of known HTIDs with which to work ...

In [3]:
# Get list of non-null HTIDs
vols = list(meta[~meta.HATHI_ID.isnull()].HATHI_ID.unique())
print('Volumes with HTID:', len(vols), "(of %s total;"%meta.TITLE.count(), 
      '{:1.1f}'.format(100*meta.HATHI_ID.count()/meta.TITLE.count()), "percent coverage)")

Volumes with HTID: 1371 (of 3525 total; 38.9 percent coverage)


## Load geo data

If `use_db` flag set to `True`, pull data from Postgres, else load from disk.

We have two different data sets to load. One uses text strings -- the places identified via NER in the source texts -- but not geolocation data. This is more fine-grained, but doesn't know anything about geographic relations. "South Bend," "Indianapolis," and "Indiana" are three records as like (or unlike) as "Chicago," "Afghanistan," and "Pacific Ocean."

The second data set relies on geolocation data and collapses locations into high-level areas (states, countries, and continents). It also maintains a modest number of raw strings for colloquial and non-specific locations such as "Pacific" and "New England" that aren't handled well by geolocation. This set sacrifices some accuracy (geolocation is error prone) and specificity for better relationality.

In [4]:
# Set up Postgres connection if using DB
#  Also need to set up SSH connection to server:
#   ssh mwilkens@geolit.crc.nd.edu -L 5433:localhost:5433

if use_db:
    import sqlalchemy
    from sqlalchemy import types
    from sqlalchemy.sql import text

    # Get credentials from file
    try:
        pg_username, pg_password = open(postgres_credentials).read().strip().split(' ')
    except:
        sys.exit('Cannot get Postgres credentials. Exiting.')

    # Set up engine and connect
    try:
        pg_engine_string = 'postgresql+psycopg2://'+pg_username+':'+pg_password+'@'+postgres_server+':'+postgres_port+'/'+postgres_db_name
        pg_engine = sqlalchemy.create_engine(pg_engine_string, echo=False)
        #pg_metadata = sqlalchemy.MetaData()
        pg_conn = pg_engine.connect()
        print("Postgres connection established.")
    except:
        sys.exit('Cannot make initial connection to Postgres database. Aborting.')
    
    # Raw strings
    query_strings = text("SELECT htid, text_string "
                          "FROM txtdata_incopyright "
                          "WHERE htid IN :s")
    result_strings = pg_conn.execute(query_strings, s=tuple(vols))
    output_strings = result_strings.fetchall()
    print("Number of raw string records found:", len(output_strings))
    data_strings = pd.DataFrame(output_strings)
    data_strings.columns = ['htid', 'text']
    # Save output to file for later use
    data_strings.to_csv(os.path.join(results_dir, 'location_strings.tsv'), 
                        sep='\t', index=False)
    
    # Geo data
    query_geo = text("SELECT htid, occurs, continent, country_short, admin_1_short, locality, text_string "
              "FROM results_full "
              "WHERE htid IN :s")
    result_geo = pg_conn.execute(query_geo, s=tuple(vols))
    output_geo = result_geo.fetchall()
    print("Number of geo data records found:", len(output_geo))
    data_geo = pd.DataFrame(output_geo)
    data_geo.columns = ['htid', 'occurs', 'continent', 'country', 'admin_1', 'locality', 'text_string']
    # Save output to file for later use
    data_geo.to_csv(os.path.join(results_dir, 'location_geodata.tsv'), 
                    sep='\t', index=False)
    pg_conn.close()
else: # Read data from disk if not using DB
    data_strings = pd.read_csv(os.path.join(results_dir, 'location_strings.tsv'), sep='\t')
    data_geo = pd.read_csv(os.path.join(results_dir, 'location_geodata.tsv'), sep='\t')

## Using raw NER text strings

In [5]:
# Look at the data
data_strings.head()

,htid,text
0,mdp.39015038551183,Square
1,mdp.39015038551183,Goggleye Lake
2,mdp.39015038551183,Rome
3,mdp.39015038551183,Decatur
4,mdp.39015038551183,Canadian River


We need this data in the form of two lists:

* HTIDs, one per volume
* Strings used as locations in each volume, each as a tokenized list

In [7]:
g = data_strings.groupby('htid')
translator = str.maketrans('\n', ' ', string.punctuation) # to remove newlines in string data
htids_strings = []
geostrings = []
for name, group in g:
    htids_strings.append(name)
    clean_strings = []
    for i in list(group.text):
        clean_strings.append(i.translate(translator).lower()) # remove \n and lowercase
    geostrings.append(clean_strings)

In [8]:
# Glance at the data
print("Dimensions of our lists:", len(htids_strings), len(geostrings))
print("\nThe first record in each list:")
print(htids_strings[0])
print(geostrings[0])

Dimensions of our lists: 1333 1333

The first record in each list:
inu.30000001734023
['los angeles', 'oconnell bridge', 'south america', 'invara', 'bayswater', 'connemara', 'goolin', 'blanchardstown', 'rumania', 'oberammergau', 'n manchester', 'palermo', 'mass', 'brazil', 'astrakhan', 'clontarf', 'vienna', 'killarney', 'movita', 'hong kong', 'dolier street', 'canada', 'new york', 'dawson street', 'gresham', 'sicily', 'south america', 'dun laoghaire', 'baba', 'oranmore', 'mass', 'harcourt street', 'soho', 'bavaria', 'nenagh', 'caithleen', 'africa', 'limerick', 'molesworth street', 'tipperary', 'europe', 'china', 'india', 'new england', 'richmond hospital', 'golden marie', 'maryborough', 'america', 'grafton street', 'west of ireland', 'honolulu', 'grangegorman', 'california', 'galway', 'london', 'killaloe', 'mount melleray', 'dawson street', 'oconnell street', 'dublin', 'piccadilly', 'ireland', 'united states of america', 'cobh', 'spain', 'england', 'toronto', 'liverpool', 'pacific', 'c

Use `sklearn` to turn our string data into a doc-term matrix. IDF weighting is optional and turned on here.

Note that we bypass preprocessing and tokenization, since that's already handled for us by the orignal NER and by our work above.

In [9]:
# define vectorizer parameters
tfidf_vectorizer_preprocessed = TfidfVectorizer(max_df=1.0, min_df=2, use_idf=True, norm = 'l2',
                                   lowercase = True, analyzer = 'word', 
                                   preprocessor=lambda x: x, tokenizer=lambda x: x)
tfidf_matrix_from_strings = tfidf_vectorizer_preprocessed.fit_transform(geostrings) #fit the vectorizer

# Get the vocabulary for later use
geo_terms_from_strings = tfidf_vectorizer_preprocessed.get_feature_names()
print("Dimenions of the doc-term matrix:", tfidf_matrix_from_strings.shape)

Dimenions of the doc-term matrix: (1333, 14029)


## Using geolocation data

Use continent-country-state data rather than raw strings. Should give better sense of general orientation, but contains more errors and has no data for some locations.

In [10]:
# Glance at the data
data_geo.head()

,htid,occurs,continent,country,admin_1,locality,text_string
0,inu.30000001734023,1,Africa,NaN,NaN,NaN,Africa
1,inu.30000001734023,10,NaN,US,NaN,NaN,America
2,inu.30000001734023,1,NaN,IE,Dublin,Dublin,Amiens Street
3,inu.30000001734023,1,NaN,RU,Astrakhan Oblast,Astrakhan,Astrakhan
4,inu.30000001734023,1,NaN,US,TX,Austin,Austin


### 'Geo words'

We want to aggregate locations by administrative area at a pretty high level. We collapse all locations into their country, except:

* For US and GB, collapse to "`admin_1`" level (US state, GB country, e.g., 'England' or 'Scotland')
* For continent references, keep as is. Note that we do not have continent data for locations that are not simple continent references, e.g., 'Africa' is labeled as a continent, but 'Nigeria' has no continent data. Not a problem, just something to know.
* For locations that are hard to collapse appropriately ('Pacific Ocean', 'Danube'), keep as is. Would be nice to be able to get continent or country data for some of these, but it doesn't exist in our sources. This is a smallish issue, since the total number of records in this class is small.

To do all this, we build a 'geoword' for each location. This 'geoword' is just the continent, country, and/or state label as appropriate, except for the small number of original text strings for the difficult cases. We then assemble the geowords into pseudodocuments to process in the same way we did with the raw strings.

In [11]:
specials = ('US', 'GB') # Countries to include admin_1

# Set geoword to continent, country, or country-admin_1 as appropriate
data_geo['geoword'] = np.where(~data_geo.continent.isnull(),
                               data_geo.continent,
                               np.where(data_geo.country.isin(specials) & ~data_geo.admin_1.isnull(),
                                   data_geo.country+'-'+data_geo.admin_1,
                                   data_geo.country)
                              )
# Set missing geowords to text string (mostly, e.g., 'New England', 'Mediterranean', etc.)
data_geo['geoword'] = np.where(~data_geo.geoword.isnull(),
                               data_geo.geoword,
                               data_geo.text_string.apply(lambda x: x.translate(translator))
                              )
data_geo.head(n=10)

,htid,occurs,continent,country,admin_1,locality,text_string,geoword
0,inu.30000001734023,1,Africa,NaN,NaN,NaN,Africa,Africa
1,inu.30000001734023,10,NaN,US,NaN,NaN,America,US
2,inu.30000001734023,1,NaN,IE,Dublin,Dublin,Amiens Street,IE
3,inu.30000001734023,1,NaN,RU,Astrakhan Oblast,Astrakhan,Astrakhan,RU
4,inu.30000001734023,1,NaN,US,TX,Austin,Austin,US-TX
5,inu.30000001734023,1,NaN,US,MA,Worcester,Baba,US-MA
6,inu.30000001734023,1,NaN,IE,Galway,Ballinasloe,Ballinasloe,IE
7,inu.30000001734023,1,NaN,DE,BY,NaN,Bavaria,DE
8,inu.30000001734023,1,NaN,US,CO,Denver,Bayswater,US-CO
9,inu.30000001734023,1,NaN,US,LA,Baton Rouge,Bengal,US-LA


Build lists of HTIDs and geowords. Note that we need to repeat each geoword as many times as it appears in each text (using data from the 'occurs' column).

In [12]:
gg = data_geo.groupby('htid')
htids_geo = []
geowords = []
for name, group in gg:
    htids_geo.append(name)
    geowords_list = []
    for item in group.itertuples():
        for i in range(item.occurs):
            geowords_list.append(item.geoword)
    geowords.append(geowords_list)

In [13]:
# Glance at the processed data
print(htids_geo[0])
print(geowords[0])

inu.30000001734023
['Africa', 'US', 'US', 'US', 'US', 'US', 'US', 'US', 'US', 'US', 'US', 'IE', 'RU', 'US-TX', 'US-MA', 'IE', 'DE', 'US-CO', 'US-LA', 'IE', 'IE', 'BR', 'BR', 'US-CA', 'CA', 'CN', 'CN', 'IE', 'IE', 'IE', 'IE', 'US-KY', 'IE', 'US-NC', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'GB-England', 'Europe', 'IE', 'IE', 'US-GA', 'US-VA', 'US-VA', 'US-VA', 'US-VA', 'IE', 'US-OR', 'IE', 'US-CA', 'US-CA', 'US-CA', 'HK', 'US-HI', 'IN', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'IT', 'IE', 'IE', 'IE', 'IE', 'US-PA', 'US-PA', 'US-PA', 'US-PA', 'US-PA', 'US-PA', 'US-PA', 'US-PA', 'US-PA', 'GB-England', 'GB-England', 'GB-Engl

In [14]:
# Print most frequent geowords, for reference only
total_geowords = 0
geoword_counts = {}
for i in geowords:
    for token in i:
        total_geowords += 1
        try:
            geoword_counts[token] += 1
        except:
            geoword_counts[token] = 1
top = sorted(geoword_counts, key=geoword_counts.get, reverse = True)

print("Total geowords: ", total_geowords)
print("Unique geowords:", len(geoword_counts.keys()))
print("\nMost frequently occurring geowords:")
for i in range(10):
    print(top[i], '\t', geoword_counts[top[i]])

Total geowords:  572789
Unique geowords: 831

Most frequently occurring geowords:
US-NY 	 51800
US-CA 	 41036
FR 	 25863
GB-England 	 25734
US 	 19730
IT 	 16548
US-TX 	 13426
RU 	 12844
US-FL 	 12348
US-MA 	 12175


In [15]:
# Fit the data
tfidf_matrix_from_geo_data = tfidf_vectorizer_preprocessed.fit_transform(geowords)

# Get the vocabulary
geo_terms_from_geo_data = tfidf_vectorizer_preprocessed.get_feature_names()
print("Shape of the output matrix:", tfidf_matrix_from_geo_data.shape)

Shape of the output matrix: (1333, 572)


## Use PCA to generate geo features

In [16]:
components = 100 # Number of principal components to fit
pca_strings = PCA(n_components=components)
pca_geodata = PCA(n_components=components)
strings_pca = pca_strings.fit_transform(tfidf_matrix_from_strings.toarray())
geodata_pca = pca_geodata.fit_transform(tfidf_matrix_from_geo_data.toarray())

print("Performance on text strings")
for i in range(10):
    pctvar = sum(pca_strings.explained_variance_ratio_[0:i])
    print("(%s) Sum of variance explained: %s" % (i, pctvar))
    
print("\nPerformance on geo data")
for i in range(10):
    pctvar = sum(pca_geodata.explained_variance_ratio_[0:i])
    print("(%s) Sum of variance explained: %s" % (i, pctvar))

Performance on text strings
(0) Sum of variance explained: 0
(1) Sum of variance explained: 0.00859547571077
(2) Sum of variance explained: 0.0145316134713
(3) Sum of variance explained: 0.0193796609091
(4) Sum of variance explained: 0.0236694662307
(5) Sum of variance explained: 0.0276644800575
(6) Sum of variance explained: 0.0310490030162
(7) Sum of variance explained: 0.0342711172846
(8) Sum of variance explained: 0.037355087152
(9) Sum of variance explained: 0.0403189099379

Performance on geo data
(0) Sum of variance explained: 0
(1) Sum of variance explained: 0.0836067964635
(2) Sum of variance explained: 0.148963279184
(3) Sum of variance explained: 0.196212313231
(4) Sum of variance explained: 0.234077645998
(5) Sum of variance explained: 0.263305967001
(6) Sum of variance explained: 0.290626733454
(7) Sum of variance explained: 0.312947131911
(8) Sum of variance explained: 0.334218613521
(9) Sum of variance explained: 0.35314557012


We obviously capture a lot more of the underlying variance in the geowords case. Not surprising, given that we're reducing from 572 features rather than 14,029 in the strings case.

Could consider using other clustering/distance/dimensionality reduction methods to generate geo features if desired. Not pursuing at the moment.

### Examine loadings

In [17]:
# Get PC loadings
loadings_strings = pca_strings.components_
loadings_strings_df = pd.DataFrame(loadings_strings.T, index=geo_terms_from_strings)
loadings_strings_df.to_csv(os.path.join(results_dir, 'loadings_strings.tsv'), sep='\t')

loadings_geodata = pca_geodata.components_
loadings_geodata_df = pd.DataFrame(loadings_geodata.T, index=geo_terms_from_geo_data)
loadings_geodata_df.to_csv(os.path.join(results_dir, 'loadings_geodata.tsv'), sep='\t')

# show top n load items for first 10 PCs
print("Top loadings for first ten PCs of string data")
for i in range(10):
    print("PC %s:"%i, loadings_strings_df[i].nlargest(15).index.values)
    
print("\nTop loadings for first ten PCs of geo data")
for i in range(10):
    print("PC %s:"%i, loadings_geodata_df[i].nlargest(15).index.values)

Top loadings for first ten PCs of string data
PC 0: ['new york city' 'la' 'los angeles' 'san francisco' 'new jersey' 'florida'
 'chicago' 'california' 'broadway' 'manhattan' 'connecticut' 'arizona'
 'texas' 'ohio' 'new orleans']
PC 1: ['fifth avenue' 'central park' 'long island' 'manhattan' 'park avenue'
 'broadway' 'brooklyn' 'east river' 'lower east side' 'bronx' 'queens'
 'west side' 'central park west' 'times square' 'east side']
PC 2: ['san francisco' 'la' 'los angeles' 'san diego' 'santa monica' 'hong kong'
 'beverly hills' 'santa barbara' 'pacific' 'sacramento' 'moscow'
 'soviet union' 'hawaii' 'southern california' 'asia']
PC 3: ['new orleans' 'mississippi' 'alabama' 'tennessee' 'south' 'north carolina'
 'memphis' 'georgia' 'south carolina' 'kentucky' 'arkansas' 'atlanta'
 'louisiana' 'soviet union' 'dc']
PC 4: ['ireland' 'scotland' 'cambridge' 'oxford' 'britain' 'wales' 'dublin'
 'victoria' 'london' 'oxford street' 'brighton' 'new england' 'yorkshire'
 'great britain' 'manches

In [18]:
# Build and save results frames
string_features = pd.DataFrame(strings_pca, index=htids_strings)
string_features.to_csv(os.path.join(results_dir, 'features_strings.csv'))
geodata_features = pd.DataFrame(geodata_pca, index=htids_geo)
geodata_features.to_csv(os.path.join(results_dir, 'features_geodata.csv'))

## Putting it together

Presumably want these features for use with other data. Below is a quick example of joining the output frame here to the existing metadata frame. Same operation would be used to join other feature data stored in other frames.

Couple of notes:

* Example below assumes you *don't* want all the PCs. It's limited to the first three. Adjust as desired.
* The example join ignores rows in the metadata frame that do not have an HTID on which to join. This is probably desired behavior, but be aware of it. To retain (with attendant NaNs in the result, change `how='right'` to `how='left'`.

In [19]:
# Example of hooking up these results with other meta/feature data
pcs_to_use = 3 # Keep this many PCs in our integrated data
data_to_join = geodata_features.loc[:,0:(pcs_to_use-1)]
integrated_results = meta.join(data_to_join, on='HATHI_ID', how='right')

print("Compare data set sizes:")
print("Volumes with geo data:", len(geodata_features.index))
print("Volumes in original metadata:", len(meta.index))
print("Volumes in integrated output:", len(integrated_results.index))
integrated_results.head()

Compare data set sizes:
Volumes with geo data: 1333
Volumes in original metadata: 3525
Volumes in integrated output: 1333


,TITLE,AUTHOR,HATHI_ID,PUBLISHER,YEAR,GENDER,RACE,NATIONALITY,GENRE,YEAR_CORRECT,0,1,2
19,Sunrise to sunset / | $c: [by] Samuel Hopkins ...,"Adams, Samuel Hopkins,",uc1.$b86201,New York|Random House|1950,1950,M,Caucasian,American,ROM,1950,0.504608,-0.137849,0.029980
20,Wait for the dawn | $c: [by] Martha Albrand [p...,"Albrand, Martha.",uc1.$b120100,New York|Random House|1950,1950,F,Caucasian,American,ROM,1950,0.055164,0.190282,-0.034396
21,"The delicate prey, | and other stories.","Bowles, Paul,",mdp.39015054095990,New York|Random House|1950,1950,M,Caucasian,American,LIT,1950,0.009584,0.051127,-0.050132
22,"The watchful gods, | and other stories.","Clark, Walter Van Tilburg,",mdp.39015013110138,New York|Random House|1950,1950,M,white,American,LIT,1950,-0.062978,0.137239,-0.071700
23,Collected stories / | $c: William Faulkner.,"Faulkner, William,",mdp.39015000689193,New York|Random House|1950.,1950,male,white,american,LIT,1950,-0.086734,-0.111916,0.024281
